In [4]:
!pip install tensorflow keras numpy

  Using cached tensorflow-2.19.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (644.8 MB)

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

2025-06-22 19:48:32.761049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750614512.914439    2838 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750614512.978249    2838 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750614513.282093    2838 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750614513.282117    2838 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750614513.282119    2838 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import glob

all_text = ""
for fname in sorted(glob.glob("dataset/*.txt")):
    with open(fname, encoding="utf-8") as f:
        all_text += f.read() + "\n"

print(f"Total corpus length: {len(all_text):,} characters")

Total corpus length: 33,631,482 characters


In [3]:
vocab = sorted(set(all_text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = {i: u for i, u in enumerate(vocab)}

print(f"Unique characters: {len(vocab)}")

Unique characters: 2123


In [ ]:
import numpy as np
from keras.utils import Sequence

class CharSequenceGenerator(Sequence):
    def __init__(self, text, seq_length, batch_size, vocab, step=1):
        self.text = text
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.vocab = vocab
        self.char2idx = {c: i for i, c in enumerate(vocab)}
        self.indices = list(range(0, len(text) - seq_length - 1, step))

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        x = np.zeros((self.batch_size, self.seq_length, len(self.vocab)), dtype=np.bool_)
        y = np.zeros((self.batch_size, len(self.vocab)), dtype=np.bool_)
        for i in range(self.batch_size):
            start = self.indices[idx * self.batch_size + i]
            seq = self.text[start:start + self.seq_length]
            next_char = self.text[start + self.seq_length]
            for t, char in enumerate(seq):
                x[i, t, self.char2idx[char]] = 1
            y[i, self.char2idx[next_char]] = 1
        return x, y

In [12]:
SEQ_LENGTH = 100
BATCH_SIZE = 256
generator = CharSequenceGenerator(all_text, SEQ_LENGTH, BATCH_SIZE, vocab, step=10)

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(256, input_shape=(SEQ_LENGTH, len(vocab)), return_sequences=True),
    Dropout(0.2),
    LSTM(256),
    Dropout(0.2),
    Dense(len(vocab), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 100, 256)       │     2,437,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2123)           │       545,611 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,508,043 (13.38 MB)

 Trainable params: 3,508,043 (13.38 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

EPOCHS = 20

callbacks = [
    ModelCheckpoint("lstm_textgen_best.h5", save_best_only=True),
    EarlyStopping(patience=3, restore_best_weights=True)
]

model.fit(generator, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/20


/home/simon/Documents/python/autocorrect-linux/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 7998/13137 ━━━━━━━━━━━━━━━━━━━━ 11:26 134ms/step - loss: 2.8984